## Pré-Processamento de Dados

In [68]:
import pandas as pd
import json
import sys
sys.path.append('lib')
import databaseinsert

In [69]:
df = pd.read_json("./datas/tweet.json", lines = True)
df.head(5)

,created_at,likes_count,screen_name,tweet
0,2021-12-16 23:23:06,186,bbcbrasil,"Para especialistas, suicídios de donas de casa..."
1,2021-12-14 15:30:17,48183,CrimesReais,"Assédio, violência, envenenamento, drogas ilíc..."
2,2021-12-17 14:22:38,7,mdhbrasil,"Promover a autonomia financeira das mulheres, ..."
3,2021-12-17 18:55:07,3,policiacivilrs,"Em Bagé, homem de 43 anos é preso, preventivam..."
4,2021-12-17 18:50:19,0,nochldlefbehind,"@aychawane eu achei uma porcaria, real. projet..."


In [70]:
#pip install nltk

In [71]:
# NLKT - Natural Language Toolkit
import nltk
#nltk.download()

In [72]:
import re
from bs4 import BeautifulSoup
from nltk.tokenize import TweetTokenizer

In [73]:
# Funções de pré-processamento (limpeza de dados)
tweet_tokenizer = TweetTokenizer()

def DataCleansing(instance):
  # Remove caracteres, artificios indesejados (links, virgula, pontuação, etc...)
  instance = re.sub(r"http\S+", "", instance).lower().replace('.', '').replace(';', '').replace('-', '').replace(':', '').replace(')', '')
  stopwords = set(nltk.corpus.stopwords.words('portuguese'))
  words = [i for i in instance.split() if not i in stopwords]
  return (" ".join(words))

def PreprocessTweet(tweet):
  tweet = BeautifulSoup(tweet, "html.parser").get_text()
  tweet = re.sub(r"[^a-zA-Zà-úÀ-Ú0-9]+", " ", tweet.lower())
  return tweet

In [74]:
# Realizando a limpeza dos dados
tweets = [DataCleansing(i) for i in df.tweet]

In [75]:
df['clean_data'] = tweets

In [88]:
df['preprocessed_data'] = [PreprocessTweet(tweet) for tweet in df.clean_data]
df_clean = df.drop(['clean_data'], axis=1)
df_clean.head()

,created_at,likes_count,screen_name,tweet,preprocessed_data
0,2021-12-16 23:23:06,186,bbcbrasil,"Para especialistas, suicídios de donas de casa...",especialistas suicídios donas casa índia relac...
1,2021-12-14 15:30:17,48183,CrimesReais,"Assédio, violência, envenenamento, drogas ilíc...",assédio violência envenenamento drogas ilícita...
2,2021-12-17 14:22:38,7,mdhbrasil,"Promover a autonomia financeira das mulheres, ...",promover autonomia financeira mulheres investi...
3,2021-12-17 18:55:07,3,policiacivilrs,"Em Bagé, homem de 43 anos é preso, preventivam...",bagé homem 43 anos preso preventivamente violê...
4,2021-12-17 18:50:19,0,nochldlefbehind,"@aychawane eu achei uma porcaria, real. projet...",aychawane achei porcaria real projeto cheio h...


## Processamento de Dataframe
### - Contagem de quantidade de Palavras

  Identificar as palavras que mais são usadas ou são associadas quando o tema é violência doméstica assédio e feminicídio.

In [77]:
# Importação de biblioteca para contar o número de vezes que uma palavra ocorre
from sklearn.feature_extraction.text import CountVectorizer

In [78]:
cv = CountVectorizer()
counts = cv.fit_transform(df.preprocessed_data)

In [79]:
# Captura as palavras de todos os tweets
cv.get_feature_names()

['001',
 '0genos',
 '12',
 '124',
 '14',
 '144',
 '148',
 '16',
 '19',
 '1slappy',
 '20',
 '202',
 '2020',
 '2021',
 '21',
 '24',
 '25',
 '26',
 '290',
 '43',
 '50',
 '58',
 '800',
 '90daysfiance',
 '90diasparacasar',
 'abalada',
 'abrigos',
 'abusivo',
 'abuso',
 'abusos',
 'acaba',
 'acabam',
 'aceita',
 'aceitam',
 'acertando',
 'achei',
 'acidentes',
 'acompanha',
 'acompanhar',
 'acontece',
 'acontecendo',
 'acordei',
 'acredito',
 'acusado',
 'acusados',
 'acusação',
 'adam',
 'administrativo',
 'admitido',
 'adolescente',
 'adorava',
 'adzingamesgay',
 'afastados',
 'afastamento',
 'afazenda',
 'afirma',
 'after',
 'agir',
 'agora',
 'agredido',
 'agredissem',
 'agressor',
 'agressores',
 'agressão',
 'agressões',
 'aguentar',
 'ainda',
 'ajuda',
 'ajudam',
 'ajudar',
 'alegações',
 'alexilapas',
 'alfa',
 'alguem',
 'alguém',
 'alternativas',
 'alvoroço',
 'além',
 'amaral',
 'amber',
 'ameaça',
 'ameaças',
 'amigos',
 'amo',
 'amor',
 'amorosas',
 'amoroso',
 'and',
 'andrea',

In [80]:
counts.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [81]:
word_count = pd.DataFrame(cv.get_feature_names(), columns = ['word'])
word_count['count'] = counts.sum(axis=0).tolist()[0]
word_count = word_count.sort_values('count', ascending = False).reset_index(drop = True)

In [82]:
words = [index for index, row in word_count.iterrows() if row['word'] == 'violência' or row['word'] == 'doméstica' or row['word'] == 'pra' or row['word'] == 'day' or row['word'] == 'quase' or row['word'] == 'pq' or row['word'] == 'sobre' or row['word'] == 'francisco' or row['word'] == 'papa' or row['word'] == 'mulher']
df_wordcount = word_count.drop(words)
df_wordcount.reset_index(inplace=True, drop=True)
df_wordcount.head(9)

,word,count
0,mulheres,25
1,contra,20
2,vítima,18
3,anos,18
4,assédio,14
5,violencia,12
6,hoje,10
7,cara,9
8,caso,9


In [83]:
word_ranking = df_wordcount.set_index('word').T.to_dict('list')
word_ranking

{'mulheres': [25],
 'contra': [20],
 'vítima': [18],
 'anos': [18],
 'assédio': [14],
 'violencia': [12],
 'hoje': [10],
 'cara': [9],
 'caso': [9],
 'domestica': [9],
 'justiça': [8],
 'satânica': [8],
 'polícia': [8],
 'todas': [7],
 'gente': [7],
 'preso': [7],
 'durante': [6],
 'dois': [6],
 'fez': [6],
 'fortaleza': [6],
 'por': [6],
 'boicotar': [6],
 'novo': [6],
 'tipo': [6],
 'afirma': [6],
 'trabalho': [6],
 'elogio': [6],
 'fazenda': [6],
 'homens': [6],
 'feminicídio': [6],
 'flagrante': [6],
 'brothers': [6],
 'nego': [6],
 'mello': [6],
 'deve': [6],
 'defender': [6],
 'âmbito': [6],
 'civil': [6],
 'livro': [6],
 'nada': [6],
 'lindain10572488': [6],
 'cantor': [6],
 'warner': [6],
 '25': [5],
 'sido': [5],
 'existem': [5],
 'ser': [5],
 'sofreu': [5],
 '20': [5],
 'moral': [5],
 'vai': [5],
 'homem': [5],
 'autoridades': [5],
 'coisa': [5],
 'enquanto': [5],
 'noite': [5],
 'situação': [4],
 'com': [4],
 'combate': [4],
 'clássico': [4],
 'dia': [4],
 'apoio': [4],
 'di

In [84]:
databaseinsert.connect_firebase()

for index, row in df_wordcount.iterrows():
  if index + 1 > 50:
    break
  elif index + 1 <= 50:

    data = {"word": row['word'],
                  "count": row['count'],
                  "position": index + 1 
            }
    position = data['position'] - 1
    databaseinsert.insert_data(data, 'word_ranking', position)